In [46]:
from location import load_all_locations, calculate_distance_components

locations = load_all_locations()
len(locations)


l_names = [l.name for l in locations]
l_dict = {name: id for id, name in enumerate(l_names)}

locations_dict = {loc.name: loc for loc in locations}
for loc in locations:
    loc.examples = list(loc.tiles.keys())[0:3]
locations[l_dict['Netherlands port']].examples = [
(51.939564, 4.065469),
(51.939564, 4.072755),
(51.939564, 4.138326),
]
locations[l_dict['Everglade safari']].examples = [
(25.665964, -80.608233),
(25.670456, -80.548431),
(25.670456, -80.588299),
]
print(l_names)
locations[l_dict['Solar farm']].examples = [
    list(locations[l_dict['Solar farm']].tiles.keys())[5],
    list(locations[l_dict['Solar farm']].tiles.keys())[13],
    list(locations[l_dict['Solar farm']].tiles.keys())[3],
    list(locations[l_dict['Solar farm']].tiles.keys())[9],
]

locations[l_dict['Parking lot']].examples = [
    list(locations[l_dict['Parking lot']].tiles.keys())[0],
    list(locations[l_dict['Parking lot']].tiles.keys())[15],
    list(locations[l_dict['Parking lot']].tiles.keys())[25]
]

print(l_names)
location = locations[2]
print(calculate_distance_components(location.top_left, location.bottom_right))

['Parking lot', 'Solar farm', 'Netherlands port', 'Everglade safari']
['Parking lot', 'Solar farm', 'Netherlands port', 'Everglade safari']
(5319.067909084073, 10488.58647579547)


In [47]:
from PIL import Image
import io
import os
import base64
import dotenv

dotenv.load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
def pil_to_base64(image, format="PNG"):
    """
    Convert a PIL image to a Base64-encoded string.

    Args:
        image (PIL.Image.Image): The PIL image to convert.
        format (str): The format to save the image (e.g., "PNG", "JPEG").

    Returns:
        str: Base64-encoded image string.
    """
    # Create a bytes buffer
    buffer = io.BytesIO()
    
    # Save the image to the buffer in the specified format
    image.save(buffer, format=format)
    
    # Get the byte data from the buffer
    buffer.seek(0)
    image_bytes = buffer.read()
    
    # Encode the byte data to Base64
    base64_image = base64.b64encode(image_bytes).decode("utf-8")
    
    return base64_image


In [48]:

css_style = """
.container {
    display: flex;
    flex-direction: column;
    align-items: center;
    margin: 20px auto;
}
.row {
    display: flex;
    flex-direction: row;
    align-items: center;
    width: 80%;
    margin-bottom: 10px;
    border: 1px solid #ccc;
    padding: 10px;
    border-radius: 5px;
    box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.1);
}
.index {
    flex: 1;
    text-align: center;
    font-weight: bold;
    color: #333;
}
.image {
    flex: 3;
    text-align: center;
}

img {
    max-width: 100%;
    height: auto;
    border-radius: 5px;
}
"""

In [49]:
from bot.rschatgpt import initilize_bot, analyze_image


#bot = RSChatGPT(gpt_name="gpt-4o-2024-11-20")
def process_individual(prompt, image, bot, threshold=.5, resolution=0.1): 

    # Save the image to a temporary file
    print(bot)
    result_list, image_out = analyze_image(bot, image, prompt, threshold, resolution)
    
    #response = bot.run_text(prompt, result_list)
    return result_list, image, image_out


In [50]:
from __future__ import annotations

from time import sleep
ex_prompt1 = "Count ships!"
ex_prompt2 = "Count ships! Show how you did it."

def process_individually(prompt: str, images: list[Image.Image], pixels_per_meter:float, bot, threshold) -> list:
    return [process_individual(prompt, image, bot, threshold, pixels_per_meter) for image in images]

def process_examples(prompt, location, bot, threshold=.5):
    #location.create_example_images() # to create random examples
    example_images = [location.tiles[example] for example in location.examples]
    return process_individually(prompt, example_images, location.meters_per_pixel, bot, threshold)

In [53]:

import os
from pathlib import Path
%load_ext autoreload
%autoreload 2
import base64
import gradio as gr
from time import sleep

# Specify your folder containing the images
image_folder = "img/"


# Load image file paths
image_paths = sorted(Path(image_folder).glob("*.png")) + sorted(Path(image_folder).glob("*.jpg"))
image_data = [(index, str(path)) for index, path in enumerate(image_paths)]

# CSS to style the layout

# Function to generate HTML content for images


css_style = """
.row_s {
    display: flex;
    flex-direction: row;
    align-items: center;
    width: 100%;
    margin-bottom: 30px;
    gap: 20px;
    justify-content: space-between; /* Adjust spacing behavior */
}
.index {
    flex: 1;
    text-align: center;
    font-weight: bold;
    color: #333;
    font-size: 1.5em;
}
.image {
    width: 40%;
    height: auto; 
    display: block; /* Center-align image inside a parent element if needed */
}
#gr-header {
    font-size: 4em !important; /* Force the size to apply */
    font-weight: bold;
    margin-bottom: 20px;
    text-align: center; 
    color: #0056b3;
}
body {
    width: 100%;
    font-family: 'Arial', sans-serif; /* Modern font */
    color: #333; /* Dark text for readability */
    margin: 0;
    padding: 0;
}

.gradio-container {
    padding: 20px;
    background-color: #f9f9f9; /* Light gray background for contrast */
}

h1, h2, h3, h4, h5, h6 {
    color: #0056b3; /* Accent color for headers */
    font-weight: bold;
}

.gr-dropdown, .gr-input {
    margin-bottom: 15px;
    padding: 8px;
    border-radius: 5px;
    border: 1px solid #ced4da;
    font-size: 0.95em;
    line-height: 1.2em;
    background-color: #ffffff;
}

.gr-html {
    font-size: 1em;
    color: #333;
    margin-top: 10px;
}

.gr-chatbot {
    max-height: 400px; 
    overflow-y: auto;
    border: 1px solid #ced4da;
    padding: 10px;
    border-radius: 8px;
    background: #ffffff; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    }



"""

js = """
function createGradioAnimation() {
    var container = document.createElement('div');
    container.id = 'gradio-animation';
    container.style.fontSize = '2em';
    container.style.fontWeight = 'bold';
    container.style.textAlign = 'center';
    container.style.marginBottom = '20px';

    var text = 'Welcome to next level Satellite image processing!';
    for (var i = 0; i < text.length; i++) {
        (function(i){
            setTimeout(function(){
                var letter = document.createElement('span');
                letter.style.opacity = '0';
                letter.style.transition = 'opacity 0.1s';
                letter.innerText = text[i];

                container.appendChild(letter);

                setTimeout(function() {
                    letter.style.opacity = '1';
                }, 20);
            }, i * 100);
        })(i);
    }

    var gradioContainer = document.querySelector('.gradio-container');
    gradioContainer.insertBefore(container, gradioContainer.firstChild);

    return 'Animation created';
}
"""


# Create the Gradio app
with gr.Blocks(css=css_style, theme=gr.themes.Base(), js=js) as demo:
    with gr.Row():
        
        with gr.Column(scale=2):
            with gr.Row():
                # gr.Button("Create a new location")
                # TODO Preselect port Amsterdam
                location_selection = gr.Dropdown(
                    value=None,
                    choices=list(locations_dict.keys()),
                    label="Recent Locations",
                    interactive=True,
                    elem_classes="gr-dropdown"  # Class added
                    
                )
                prompt_input = gr.Text('', label="Prompt", elem_classes="gr-input")  # Class added)
            with gr.Row():
                gr.Markdown("Selected Area")
                gr.Markdown("Description about the selected area")
            with gr.Row():
                image_display = gr.Image(label=None, interactive=False, width=500, height=500, elem_classes="image")
                with gr.Column():
                    number_of_tiles = gr.Text("1000x1000", label="Number of tiles", visible=False)
                    real_size = gr.Text("1000x1000", label="Size (meters)", visible=False)
            output = gr.HTML(value="", elem_classes="gr-html")
            #image_display = gr.Image(label=None, interactive=False, width=500, height=500, elem_classes="image")


            threshold_value = 0.5
            # Add an empty HTML element
            #with gr.Row(elem_classes="gr-slider-container"):
            def update_threshold(value):
                global threshold_value
                threshold_value = value  # Update the global value
            threshold_slider = gr.Slider(0, 1, value=0.5, step=0.1, label="Threshold", elem_classes="gr-slider", info="Choose a threshold between 0 and 1", interactive=True)
            threshold_slider.change(update_threshold, inputs=threshold_slider, outputs=[])

            annotate_examples_btn = gr.Button("Annotate Examples", interactive=False)
            output = gr.HTML(value="<div></div>")
            

            # Add a button to load images
            

            # Update the HTML content when the button is pressed

            submit_all_button = gr.Button("Run", visible=False)
        with gr.Column():
            chat_history = [
             #   {"role": "user", "content": "Hello!"},
             #   {"role": "assistant", "content": "Hi there! How can I assist you today?"},
            ]
            def get_chat_history():
                return chat_history
            # Gradio Blocks Interface
            chatbot = gr.Chatbot(type="messages", value=get_chat_history(), show_label=False)  # Load initial chat history


            # Function to refresh chat: clear history and reload with updated messages

            # Refresh Button
            #refresh_button = gr.Button("Refresh Chat")
            # refresh_button.click(add_to_chat, inputs=[], outputs=chatbot)


    ### Events
    def add_to_chat(messages):
        global chat_history
        for role, message in messages:
            chat_history.append({"role": role, "content": message})

        return gr.update(value=chat_history)  # Update the chatbot with the new history


    def fetch_examples(prompt_input: str, location_name: str, progress=gr.Progress()):
        bot = initilize_bot(prompt_input)
        #gr.Button("Run", visible=True)
        location = locations_dict[location_name]
        progress(0.005, desc="Starting")
        api_response = process_examples(prompt_input, location, bot, threshold_value)
        
        assert len(api_response[0]) == 3

        html_content = '<div class="container">'
        for answer, example_image, annotated_image in progress.tqdm(api_response, desc="Processing example images"):
            gr.HTML(value="<div></div>")
                #with open(image_path, "rb") as img_file:
                #    base64_image = base64.b64encode(img_file.read()).decode("utf-8")
            base64_image = pil_to_base64(example_image)    
            html_content += f"""
            <div class="row_s">
                <div class="image"><img src="data:image/png;base64,{base64_image}"></div>
                {
                    "" if annotated_image is None else f'<div class="image"><img src="data:image/png;base64,{pil_to_base64(annotated_image)}"></div>'
                }

                <div class="index">{answer}</div>
            </div>
            """

        html_content += "</div>"
        return html_content, gr.Button(f"Run query on all {len(location.tiles)} tiles", visible=True)
    # submit_button.click(fetch_examples, inputs=[prompt_input, location_selection], outputs=output)

    annotate_examples_btn.click(fetch_examples, inputs=[prompt_input, location_selection], outputs=[output, submit_all_button])


    def show_button():
        return gr.HTML('')
    submit_all_button.click(show_button, inputs=[], outputs=[output])

    def submit_all(prompt_input: str, location_name: str, progress=gr.Progress()):
        bot = initilize_bot(prompt_input)
        progress(0.05, desc="Starting")
        location = locations_dict[location_name]
        vals = [process_individual(prompt_input, tile, bot) for tile in progress.tqdm(list(location.tiles.values()), desc="Processing the images")]
        new_vals = [float(val[0]) for val in vals]
        
        gr.Text(f"Total_num: {sum(new_vals)}")
        gr.Text(f"Total: {vals}")
        messages = [("user", prompt_input), ("assistant", f"At location {location_name}, I see {sum(new_vals)} in total.")]
        return add_to_chat(messages)
    
    submit_all_button.click(submit_all, inputs=[prompt_input, location_selection], outputs=[chatbot])
    def hide_submit_all_button():
        return gr.Button("", visible=False)
    submit_all_button.click(hide_submit_all_button, inputs=[], outputs=[submit_all_button])

    def on_location_selected(location_name: str, prompt):
        location = locations_dict[location_name]
        should_be_interactive = prompt != "" and location_name != ""
        area = calculate_distance_components(location.top_left, location.bottom_right)
        real_size_str = f"{round(area[0])} x {round(area[1])}"
        return location.thumbnail, gr.Button("Annotate Examples", interactive=should_be_interactive), gr.Text(real_size_str, label="Size (meters)", visible=True), gr.Text(str(len(location.tiles)), label="Number of tiles", visible=True)
    
    def on_prompt_input(prompt: str, location_name: str):
        #location = locations_dict[location_name]
        should_be_interactive = prompt != "" and location_name != ""
        return gr.Button("Annotate Examples", interactive=should_be_interactive)
    location_selection.change(on_location_selected, inputs=[location_selection, prompt_input], outputs=[image_display, annotate_examples_btn, real_size, number_of_tiles,])
    prompt_input.change(on_prompt_input, inputs=[location_selection, prompt_input], outputs=annotate_examples_btn)
# Run the app
demo.launch()




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Running on local URL:  http://127.0.0.1:7875

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


Detected entities:
 ['forest land']
Prompt:
 Answer the question about the image based on the given fields' areas.
Output only a single number as your output between double square brackets as, e.g [[6]].

Area of each type of field:
urban land: area = 47.36 square meters, area ratio = 0.0104
agriculture: area = 32.29 square meters, area ratio = 0.0071
rangeland: area = 62.42 square meters, area ratio = 0.0137
forest land: area = 2.15 square meters, area ratio = 0.0005
water: area = 45.20 square meters, area ratio = 0.0099
barren land: area = 4365.40 square meters, area ratio = 0.9584

Question:
What is the area of forest?

Prompt:
 Answer the question about the image based on the given fields' areas.
Output only a single number as your output between double square brackets as, e.g [[6]].

Area of each type of field:
urban land: area = 975.11 square meters, area ratio = 0.2141
agriculture: area = 19.37 square meters, area ratio = 0.0043
rangeland: area = 187.27 square meters, area ratio